In [1]:
import pandas as pd
import numpy as np
import tarfile
import os
import glob as glob
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor 
from datetime import datetime
            
import xgboost as xgb

from dask_cuda import LocalCUDACluster
from dask.distributed import Client
from dask import array as da
from dask import dataframe as dd 
import dask
import xgboost as xgb
from xgboost.dask import DaskDMatrix

import lightgbm as lgb

from tqdm import tqdm

In [28]:
print("Reading the training data")

final_train_DF = pd.DataFrame()
for i, filename in enumerate(glob.glob("../data/train_batch_*"+"colwise.pickle")):
    print(filename)
    DF = pd.read_pickle(filename)
    final_train_DF = final_train_DF.append(DF)
    print(final_train_DF.shape, DF.shape)
final_train_DF.reset_index(drop = True, inplace = True)

print("")    
print("Reading the test data")
final_test_null_DF = pd.read_pickle('../data/test_DF_null_colwise.pickle')
final_test_null_DF.reset_index(drop = True, inplace = True)


Reading the test data


In [99]:
diff_columns = final_train_DF.columns.difference(final_test_null_DF.columns)

miss_test_columns = pd.DataFrame(np.ones([final_test_null_DF.shape[1], len(diff_columns)]), columns = diff_columns)
miss_test_columns = miss_test_columns.replace(1, -999)
miss_test_columns

final_test_null_DF = pd.concat([final_test_null_DF, miss_test_columns], axis = 1)
final_test_null_DF.shape, final_train_DF.shape

((109474, 51), (12994599, 51))

In [31]:
final_train_DF.drop(['measurements'], axis =  1, inplace = True)
final_test_null_DF.drop(['measurements'], axis =  1, inplace = True)

final_train_DF.fillna(-999, inplace = True)
final_test_null_DF.fillna(-999, inplace = True)

In [35]:
dep = ['latitude', 'longitude', 'geoAltitude']
drop = ['id', 'file', 'measurements'] 
indep = final_train_DF.columns.difference(dep + drop)

final_train_DF[indep]

,aircraft,baroAltitude,numMeasurements,sensor_id_0,sensor_id_1,sensor_id_10,sensor_id_11,sensor_id_12,sensor_id_13,sensor_id_2,...,sensor_time_13,sensor_time_2,sensor_time_3,sensor_time_4,sensor_time_5,sensor_time_6,sensor_time_7,sensor_time_8,sensor_time_9,timeAtServer
0,634,5562.60,2,289.0,191.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1045.032
1,634,5562.60,2,289.0,191.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1045.032
2,634,5615.94,2,191.0,289.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1048.918
3,2197,68.58,2,101.0,440.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1050.078
4,2197,83.82,2,101.0,440.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1051.117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12994594,1729,6720.84,2,289.0,191.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,3599.946
12994595,15,11056.62,2,320.0,248.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,3599.952
12994596,163,10668.00,2,114.0,130.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,3599.952
12994597,281,11269.98,2,473.0,320.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,3599.965


# Decision Tree

In [5]:
# DT = DecisionTreeRegressor()

# final_pred_DT = {}
# for label in dep:
#     print(label)
#     DT.fit(final_train_DF[indep], final_train_DF[label])
#     DT_predict_prod = DT.predict(final_test_null_DF[indep])
#     final_pred_DT[label] = DT_predict_prod
    
    
    
# final_pred_DT['id'] = final_test_null_DF.id
# DT_submission_DF = pd.DataFrame(final_pred_DT)
# DT_submission_DF = DT_submission_DF[['id', 'latitude', 'longitude', 'geoAltitude']]


# DT_submission_DF = DT_submission_DF.groupby(['id']).agg(latitude = ('latitude', 'mean'), 
#                                                         longitude = ('longitude', 'mean'),
#                                                         geoAltitude = ('geoAltitude', 'mean')).reset_index()

# DT_submission_DF.to_csv('../output/sub_DT_5.csv', index = False)
# DT_submission_DF.shape

# Gradient Boosting

In [10]:


# GBM = GradientBoostingRegressor(n_estimators = 50)

# final_pred_GBM = {}
# for label in dep:
#     print(label)
#     GBM.fit(final_train_DF[indep], final_train_DF[label])
#     predict_prod = GBM.predict(final_test_null_DF[indep])
#     final_pred_GBM[label] = predict_prod
    

# final_pred_GBM['id'] = final_test_null_DF.id
# GBM_submission_DF = pd.DataFrame(final_pred_GBM)
# GBM_submission_DF = GBM_submission_DF[['id', 'latitude', 'longitude', 'geoAltitude']]

# GBM_submission_DF = GBM_submission_DF.groupby(['id']).agg(latitude = ('latitude', 'mean'), 
#                                                           longitude = ('longitude', 'mean'),
#                                                           geoAltitude = ('geoAltitude', 'mean')).reset_index()

# GBM_submission_DF.to_csv('../output/sub_GBM_4.csv', index = False)
# GBM_submission_DF

# XGBOOST - Multi-GPU training

In [43]:
num_rounds = 1000
params = {'objective' : 'reg:linear'
          ,'eval_metric': 'rmse'
          ,'max_depth' : 6
          ,'eta' : 0.3
          ,'subsample': 1
          ,'colsample_bytree': 1
          ,'tree_method' : 'gpu_hist'
          }

final_output = pd.DataFrame()
final_output['id'] = final_test_null_DF.id
score_dict = {}
for lab in dep[0:2]:
    with LocalCUDACluster(n_workers=2, threads_per_worker=8) as cluster:
        with Client(cluster) as client:
        
            print(lab)
            start_time = datetime.now()
            print(start_time)
            
            np.random.seed(100)
            train_local_X, valid_local_X, train_local_Y, valid_local_Y = train_test_split(final_train_DF[indep],
                                                                                          final_train_DF[lab],
                                                                                          test_size = 0.2,
                                                                                          random_state = 100)

            print("Split complete")
            print(train_local_X.shape, train_local_Y.shape, valid_local_X.shape, valid_local_Y.shape)

            print("forming the dask local set")
            train_local_X = dd.from_pandas(train_local_X, npartitions=4)
            train_local_Y = dd.from_pandas(train_local_Y, npartitions=4)
            valid_local_X = dd.from_pandas(valid_local_X, npartitions=4)
            valid_local_Y = dd.from_pandas(valid_local_Y, npartitions=4)
            
            print("forming the dask prod set")
            #train_prod_X = dd.from_pandas(final_train_DF[indep], npartitions=4)
            #train_prod_Y = dd.from_pandas(final_train_DF[dep], npartitions=4)
            test_prod_X = dd.from_pandas(final_test_null_DF[indep], npartitions=4)
            
            
            dtrain_local = DaskDMatrix(client, data = train_local_X, label = train_local_Y)
            dtest_local  = DaskDMatrix(client, data = valid_local_X, label = valid_local_Y)
            #dtrain_prod = DaskDMatrix(client, data = train_prod_X, label = train_prod_Y)
            dtest_prod = DaskDMatrix(client, data = test_prod_X)

            eval_set = [(dtrain_local,'train'), (dtest_local,'test')]

            print("Formed the dataset to to be accepted by XGBoost")

            np.random.seed(100)
            local_model = xgb.dask.train(client, 
                                         params,
                                         dtrain_local,
                                         evals = eval_set,
                                         num_boost_round = num_rounds,
                                         verbose_eval = True,
                                         early_stopping_rounds = 20
                                        )
            
            bst = local_model['booster']
            history = local_model['history']
    
            xgb_prod_predict = xgb.dask.predict(client, bst, dtest_prod)
            xgb_prod_predict = np.array(xgb_prod_predict)

#             best_iteration = local_model.best_iteration
#             best_score = local_model.best_score

#             score_dict[lab] = best_score

#             np.random.seed(100)
#             clf = xgb.train(params,
#                             dtrain_prod,
#                             num_boost_round = best_iteration,
#                             evals = eval_set,
#                             verbose_eval = True,
#                             early_stopping_rounds = 20
#                                        )

#             xgb_prod_predict = clf.predict(dtest_prod)

            final_output[lab] = xgb_prod_predict
    
            end_time = datetime.now()
            print(end_time)
            
            total_time = (end_time - start_time)
            print("The {} loop took {} mins time to complete".format(lab, total_time))
            print("")
print(score_dict)


latitude
Split complete
(30400977, 11) (30400977,) (7600245, 11) (7600245,)
forming the dask local set
forming the dask prod set
Formed the dataset to to be accepted by XGBoost

longitude
Split complete
(30400977, 11) (30400977,) (7600245, 11) (7600245,)
forming the dask local set
forming the dask prod set
Formed the dataset to to be accepted by XGBoost

{}


In [46]:
evaluation = pd.DataFrame(history['train']).rename(columns = {'rmse' : 'train_rmse'})
evaluation['test_rmse'] = history['test']['rmse']
evaluation

,train_rmse,test_rmse
0,5.942095,5.943219
1,4.252050,4.252944
2,3.101866,3.102647
3,2.337739,2.338396
4,1.850369,1.850954
...,...,...
995,0.948539,0.950452
996,0.948484,0.950396
997,0.948318,0.950236
998,0.948164,0.950079


In [51]:
final_output['geoAltitude'] = final_test_null_DF['baroAltitude']

In [52]:
XGB_submission_DF = final_output.groupby(['id']).agg(latitude = ('latitude', 'mean'), 
                                                     longitude = ('longitude', 'mean'),
                                                     geoAltitude = ('geoAltitude', 'mean')).reset_index()

XGB_submission_DF.to_csv('../output/sub_XGB_8.csv', index = False)
XGB_submission_DF

,id,latitude,longitude,geoAltitude
0,31,46.875130,5.744759,10965.18
1,79,46.807526,6.790453,11277.60
2,89,45.980431,5.776221,11582.40
3,101,48.799751,6.585725,11887.20
4,132,51.452751,0.540696,9014.46
...,...,...,...,...
109469,1836617,49.940079,5.184569,10363.20
109470,1836625,47.727474,9.688813,7886.70
109471,1836645,49.443455,6.512658,10668.00
109472,1836697,48.778038,9.945557,10713.72


In [111]:
final_output['match'] = final_output.signal_strength == final_output.best_signal_sensor

final_output = final_output.loc[final_output.match,].reset_index(drop = True)
final_output

,index,id,latitude,longitude,geoAltitude,sensor_id,signal_strength,best_signal_sensor,match
0,1,31,47.492760,6.490246,11281.096680,470.0,42.0,42.0,True
1,3,79,46.915699,6.091815,11704.291992,474.0,36.0,36.0,True
2,5,89,45.767265,6.447528,12074.622070,131.0,122.0,122.0,True
3,6,101,48.928204,7.319705,12104.121094,130.0,58.0,58.0,True
4,13,132,51.020058,-0.671965,9313.395508,294.0,70.0,70.0,True
...,...,...,...,...,...,...,...,...,...
110938,377586,1836617,49.476978,5.095135,10741.397461,414.0,36.0,36.0,True
110939,377588,1836625,47.229134,8.362603,8275.097656,147.0,92.0,92.0,True
110940,377592,1836645,50.759956,6.075030,11028.493164,398.0,103.0,103.0,True
110941,377594,1836697,48.404011,9.974515,11145.338867,598.0,93.0,93.0,True


In [113]:
final_output.id.nunique()

109474

# Xgboost on a single GPU

In [44]:
# num_rounds = 10000
# params = {'objective' : 'reg:linear'
#           ,'eval_metric': 'rmse'
#           ,'max_depth' : 6
#           ,'eta' : 0.3
#           ,'subsample': 1
#           ,'colsample_bytree': 1
#           ,'tree_method' : 'gpu_hist'
#           }

# final_output = pd.DataFrame()
# final_output['ID'] = final_test_null_DF.id
# score_dict = {}
# for lab in dep:
#     print(lab)
#     np.random.seed(100)
#     train_local_X, valid_local_X, train_local_Y, valid_local_Y = train_test_split(final_train_DF[indep],
#                                                                                   final_train_DF[lab],
#                                                                                   test_size = 0.2,
#                                                                                   random_state = 100)
    
#     print("Split complete")
#     print(train_local_X.shape, train_local_Y.shape, valid_local_X.shape, valid_local_Y.shape)
    
#     dtrain_local = xgb.DMatrix(data = train_local_X, label = train_local_Y)
#     dtest_local  = xgb.DMatrix(data = valid_local_X, label = valid_local_Y)
# #     dtrain_prod = xgb.DMatrix(data = final_train_DF[indep], label = final_train_DF[lab])
# #     dtest_prod = xgb.DMatrix(data = final_test_null_DF[indep])

#     eval_set = [(dtrain_local,'train'), (dtest_local,'test')]

#     print("Formed the dataset to tbe accepted by XGBoost")
    
#     np.random.seed(100)
#     local_model = xgb.train(params,
#                             dtrain_local,
#                             evals = eval_set,
#                             num_boost_round = num_rounds,
#                             verbose_eval = True,
#                             early_stopping_rounds = 20
#                                        )
#     xgb_prod_predict = local_model.predict(dtest_prod)

#     best_iteration = local_model.best_iteration
#     best_score = local_model.best_score

#     score_dict[lab] = best_score
    
# #     np.random.seed(100)
# #     clf = xgb.train(params,
# #                     dtrain_prod,
# #                     num_boost_round = best_iteration,
# #                     evals = eval_set,
# #                     verbose_eval = True,
# #                     early_stopping_rounds = 20
# #                                )

# #     xgb_prod_predict = clf.predict(dtest_prod)
    
#     final_output[lab] = xgb_prod_predict
#     print("")

# final_output = final_output[['ID','Depression', 'Alcohol', 'Suicide', 'Drugs']]

# print(score_dict)


In [114]:
# XGB_submission_DF = final_output.groupby(['id']).agg(latitude = ('latitude', 'median'), 
#                                                      longitude = ('longitude', 'median'),
#                                                      geoAltitude = ('geoAltitude', 'median')).reset_index()

# XGB_submission_DF.to_csv('../output/sub_XGB_7.csv', index = False)
# XGB_submission_DF

,id,latitude,longitude,geoAltitude
0,31,47.492760,6.490246,11281.096680
1,79,46.915699,6.091815,11704.291992
2,89,45.767265,6.447528,12074.622070
3,101,48.928204,7.319705,12104.121094
4,132,51.020058,-0.671965,9313.395508
...,...,...,...,...
109469,1836617,49.476978,5.095135,10741.397461
109470,1836625,47.229134,8.362603,8275.097656
109471,1836645,50.759956,6.075030,11028.493164
109472,1836697,48.404011,9.974515,11145.338867


# LightGBM

In [103]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'rmse'},
    'num_leaves': 16,
    'learning_rate': 0.3,
    'feature_fraction': 1,
    'bagging_fraction': 1,
    'bagging_freq': 1,
    'verbose': 0
}

final_output = pd.DataFrame()
final_output['id'] = final_test_null_DF.id
score_dict = {}
for lab in dep[0:2]:
    print(lab)
    np.random.seed(100)
    train_local_X, valid_local_X, train_local_Y, valid_local_Y = train_test_split(final_train_DF[indep],
                                                                                  final_train_DF[lab],
                                                                                  test_size = 0.2,
                                                                                  random_state = 100)

    lgb_train = lgb.Dataset(train_local_X, train_local_Y)
    lgb_eval = lgb.Dataset(valid_local_X, valid_local_Y, reference=lgb_train)


    num_rounds = 10000
    print('Starting training...')
    # train
    np.random.seed(100)
    lgb_model = lgb.train(params,
                          lgb_train,
                          num_boost_round=num_rounds ,
                          valid_sets=lgb_eval,
                          early_stopping_rounds=20)
    
    lgb_prod_predict = lgb_model.predict(final_test_null_DF[indep])
    final_output[lab] = lgb_prod_predict

latitude
Starting training...
[1]	valid_0's rmse: 2.37379
Training until validation scores don't improve for 20 rounds
[2]	valid_0's rmse: 1.93776
[3]	valid_0's rmse: 1.65526
[4]	valid_0's rmse: 1.49067
[5]	valid_0's rmse: 1.31517
[6]	valid_0's rmse: 1.2245
[7]	valid_0's rmse: 1.09506
[8]	valid_0's rmse: 1.04201
[9]	valid_0's rmse: 0.977646
[10]	valid_0's rmse: 0.925195
[11]	valid_0's rmse: 0.871729
[12]	valid_0's rmse: 0.842462
[13]	valid_0's rmse: 0.812972
[14]	valid_0's rmse: 0.785201
[15]	valid_0's rmse: 0.768663
[16]	valid_0's rmse: 0.751806
[17]	valid_0's rmse: 0.732457
[18]	valid_0's rmse: 0.722246
[19]	valid_0's rmse: 0.714136
[20]	valid_0's rmse: 0.699196
[21]	valid_0's rmse: 0.687032
[22]	valid_0's rmse: 0.677801
[23]	valid_0's rmse: 0.664002
[24]	valid_0's rmse: 0.658614
[25]	valid_0's rmse: 0.651652
[26]	valid_0's rmse: 0.648025
[27]	valid_0's rmse: 0.63974
[28]	valid_0's rmse: 0.636948
[29]	valid_0's rmse: 0.633237
[30]	valid_0's rmse: 0.626298
[31]	valid_0's rmse: 0.62118

[267]	valid_0's rmse: 0.48686
[268]	valid_0's rmse: 0.486732
[269]	valid_0's rmse: 0.486628
[270]	valid_0's rmse: 0.486508
[271]	valid_0's rmse: 0.486451
[272]	valid_0's rmse: 0.486352
[273]	valid_0's rmse: 0.486263
[274]	valid_0's rmse: 0.486193
[275]	valid_0's rmse: 0.486171
[276]	valid_0's rmse: 0.486145
[277]	valid_0's rmse: 0.485913
[278]	valid_0's rmse: 0.48568
[279]	valid_0's rmse: 0.485499
[280]	valid_0's rmse: 0.485354
[281]	valid_0's rmse: 0.485236
[282]	valid_0's rmse: 0.485201
[283]	valid_0's rmse: 0.485138
[284]	valid_0's rmse: 0.485053
[285]	valid_0's rmse: 0.484954
[286]	valid_0's rmse: 0.484907
[287]	valid_0's rmse: 0.484786
[288]	valid_0's rmse: 0.484678
[289]	valid_0's rmse: 0.484619
[290]	valid_0's rmse: 0.48437
[291]	valid_0's rmse: 0.48425
[292]	valid_0's rmse: 0.484173
[293]	valid_0's rmse: 0.484026
[294]	valid_0's rmse: 0.483969
[295]	valid_0's rmse: 0.483941
[296]	valid_0's rmse: 0.483772
[297]	valid_0's rmse: 0.483752
[298]	valid_0's rmse: 0.483698
[299]	valid_

[533]	valid_0's rmse: 0.463037
[534]	valid_0's rmse: 0.463
[535]	valid_0's rmse: 0.462977
[536]	valid_0's rmse: 0.46291
[537]	valid_0's rmse: 0.462756
[538]	valid_0's rmse: 0.462659
[539]	valid_0's rmse: 0.462549
[540]	valid_0's rmse: 0.462518
[541]	valid_0's rmse: 0.462473
[542]	valid_0's rmse: 0.462344
[543]	valid_0's rmse: 0.462243
[544]	valid_0's rmse: 0.462197
[545]	valid_0's rmse: 0.462122
[546]	valid_0's rmse: 0.462062
[547]	valid_0's rmse: 0.462053
[548]	valid_0's rmse: 0.461975
[549]	valid_0's rmse: 0.461848
[550]	valid_0's rmse: 0.46181
[551]	valid_0's rmse: 0.46177
[552]	valid_0's rmse: 0.461759
[553]	valid_0's rmse: 0.461731
[554]	valid_0's rmse: 0.46165
[555]	valid_0's rmse: 0.461621
[556]	valid_0's rmse: 0.46154
[557]	valid_0's rmse: 0.461505
[558]	valid_0's rmse: 0.46146
[559]	valid_0's rmse: 0.461398
[560]	valid_0's rmse: 0.461341
[561]	valid_0's rmse: 0.461305
[562]	valid_0's rmse: 0.461237
[563]	valid_0's rmse: 0.461212
[564]	valid_0's rmse: 0.461169
[565]	valid_0's r

[800]	valid_0's rmse: 0.45035
[801]	valid_0's rmse: 0.450241
[802]	valid_0's rmse: 0.450183
[803]	valid_0's rmse: 0.450152
[804]	valid_0's rmse: 0.450083
[805]	valid_0's rmse: 0.450058
[806]	valid_0's rmse: 0.450003
[807]	valid_0's rmse: 0.449951
[808]	valid_0's rmse: 0.449927
[809]	valid_0's rmse: 0.449877
[810]	valid_0's rmse: 0.449789
[811]	valid_0's rmse: 0.449682
[812]	valid_0's rmse: 0.449668
[813]	valid_0's rmse: 0.449657
[814]	valid_0's rmse: 0.449601
[815]	valid_0's rmse: 0.449565
[816]	valid_0's rmse: 0.449468
[817]	valid_0's rmse: 0.449413
[818]	valid_0's rmse: 0.449353
[819]	valid_0's rmse: 0.449296
[820]	valid_0's rmse: 0.449229
[821]	valid_0's rmse: 0.449199
[822]	valid_0's rmse: 0.449187
[823]	valid_0's rmse: 0.44917
[824]	valid_0's rmse: 0.449142
[825]	valid_0's rmse: 0.449088
[826]	valid_0's rmse: 0.449039
[827]	valid_0's rmse: 0.448985
[828]	valid_0's rmse: 0.448965
[829]	valid_0's rmse: 0.448959
[830]	valid_0's rmse: 0.448952
[831]	valid_0's rmse: 0.448891
[832]	vali

[1064]	valid_0's rmse: 0.441129
[1065]	valid_0's rmse: 0.441114
[1066]	valid_0's rmse: 0.441089
[1067]	valid_0's rmse: 0.441047
[1068]	valid_0's rmse: 0.441035
[1069]	valid_0's rmse: 0.440991
[1070]	valid_0's rmse: 0.440975
[1071]	valid_0's rmse: 0.440932
[1072]	valid_0's rmse: 0.440893
[1073]	valid_0's rmse: 0.440876
[1074]	valid_0's rmse: 0.44085
[1075]	valid_0's rmse: 0.440796
[1076]	valid_0's rmse: 0.440783
[1077]	valid_0's rmse: 0.440753
[1078]	valid_0's rmse: 0.440726
[1079]	valid_0's rmse: 0.44072
[1080]	valid_0's rmse: 0.440686
[1081]	valid_0's rmse: 0.440639
[1082]	valid_0's rmse: 0.440554
[1083]	valid_0's rmse: 0.440526
[1084]	valid_0's rmse: 0.440501
[1085]	valid_0's rmse: 0.440493
[1086]	valid_0's rmse: 0.440479
[1087]	valid_0's rmse: 0.440473
[1088]	valid_0's rmse: 0.440422
[1089]	valid_0's rmse: 0.440413
[1090]	valid_0's rmse: 0.440348
[1091]	valid_0's rmse: 0.440283
[1092]	valid_0's rmse: 0.44025
[1093]	valid_0's rmse: 0.440222
[1094]	valid_0's rmse: 0.440171
[1095]	vali

...

[8874]	valid_0's rmse: 0.570428
[8875]	valid_0's rmse: 0.570418
[8876]	valid_0's rmse: 0.570395
[8877]	valid_0's rmse: 0.570378
[8878]	valid_0's rmse: 0.570376
[8879]	valid_0's rmse: 0.570372
[8880]	valid_0's rmse: 0.570368
[8881]	valid_0's rmse: 0.570361
[8882]	valid_0's rmse: 0.570361
[8883]	valid_0's rmse: 0.570361
[8884]	valid_0's rmse: 0.570338
[8885]	valid_0's rmse: 0.570325
[8886]	valid_0's rmse: 0.570315
[8887]	valid_0's rmse: 0.5703
[8888]	valid_0's rmse: 0.570289
[8889]	valid_0's rmse: 0.570283
[8890]	valid_0's rmse: 0.570268
[8891]	valid_0's rmse: 0.570261
[8892]	valid_0's rmse: 0.570259
[8893]	valid_0's rmse: 0.570247
[8894]	valid_0's rmse: 0.570244
[8895]	valid_0's rmse: 0.57024
[8896]	valid_0's rmse: 0.570237
[8897]	valid_0's rmse: 0.57022
[8898]	valid_0's rmse: 0.570213
[8899]	valid_0's rmse: 0.570211
[8900]	valid_0's rmse: 0.570209
[8901]	valid_0's rmse: 0.570193
[8902]	valid_0's rmse: 0.570189
[8903]	valid_0's rmse: 0.570182
[8904]	valid_0's rmse: 0.57017
[8905]	valid_

[9132]	valid_0's rmse: 0.567918
[9133]	valid_0's rmse: 0.567918
[9134]	valid_0's rmse: 0.567899
[9135]	valid_0's rmse: 0.567889
[9136]	valid_0's rmse: 0.567875
[9137]	valid_0's rmse: 0.56787
[9138]	valid_0's rmse: 0.567866
[9139]	valid_0's rmse: 0.567858
[9140]	valid_0's rmse: 0.567825
[9141]	valid_0's rmse: 0.567799
[9142]	valid_0's rmse: 0.567789
[9143]	valid_0's rmse: 0.567755
[9144]	valid_0's rmse: 0.567753
[9145]	valid_0's rmse: 0.567751
[9146]	valid_0's rmse: 0.567739
[9147]	valid_0's rmse: 0.567729
[9148]	valid_0's rmse: 0.567728
[9149]	valid_0's rmse: 0.567726
[9150]	valid_0's rmse: 0.567717
[9151]	valid_0's rmse: 0.567715
[9152]	valid_0's rmse: 0.567696
[9153]	valid_0's rmse: 0.567688
[9154]	valid_0's rmse: 0.567681
[9155]	valid_0's rmse: 0.567673
[9156]	valid_0's rmse: 0.567667
[9157]	valid_0's rmse: 0.567663
[9158]	valid_0's rmse: 0.567652
[9159]	valid_0's rmse: 0.567623
[9160]	valid_0's rmse: 0.567618
[9161]	valid_0's rmse: 0.56761
[9162]	valid_0's rmse: 0.567577
[9163]	val

[9389]	valid_0's rmse: 0.565716
[9390]	valid_0's rmse: 0.565692
[9391]	valid_0's rmse: 0.565667
[9392]	valid_0's rmse: 0.565666
[9393]	valid_0's rmse: 0.565665
[9394]	valid_0's rmse: 0.565655
[9395]	valid_0's rmse: 0.565652
[9396]	valid_0's rmse: 0.565651
[9397]	valid_0's rmse: 0.565646
[9398]	valid_0's rmse: 0.565629
[9399]	valid_0's rmse: 0.565611
[9400]	valid_0's rmse: 0.565594
[9401]	valid_0's rmse: 0.565593
[9402]	valid_0's rmse: 0.565576
[9403]	valid_0's rmse: 0.565571
[9404]	valid_0's rmse: 0.565556
[9405]	valid_0's rmse: 0.565551
[9406]	valid_0's rmse: 0.565535
[9407]	valid_0's rmse: 0.565529
[9408]	valid_0's rmse: 0.565516
[9409]	valid_0's rmse: 0.565513
[9410]	valid_0's rmse: 0.565499
[9411]	valid_0's rmse: 0.565471
[9412]	valid_0's rmse: 0.565469
[9413]	valid_0's rmse: 0.565463
[9414]	valid_0's rmse: 0.565461
[9415]	valid_0's rmse: 0.565459
[9416]	valid_0's rmse: 0.565444
[9417]	valid_0's rmse: 0.565431
[9418]	valid_0's rmse: 0.565424
[9419]	valid_0's rmse: 0.565418
[9420]	v

[9646]	valid_0's rmse: 0.563326
[9647]	valid_0's rmse: 0.563325
[9648]	valid_0's rmse: 0.563316
[9649]	valid_0's rmse: 0.563308
[9650]	valid_0's rmse: 0.563304
[9651]	valid_0's rmse: 0.563295
[9652]	valid_0's rmse: 0.563287
[9653]	valid_0's rmse: 0.563271
[9654]	valid_0's rmse: 0.563261
[9655]	valid_0's rmse: 0.56326
[9656]	valid_0's rmse: 0.563248
[9657]	valid_0's rmse: 0.563236
[9658]	valid_0's rmse: 0.563228
[9659]	valid_0's rmse: 0.563228
[9660]	valid_0's rmse: 0.563212
[9661]	valid_0's rmse: 0.563208
[9662]	valid_0's rmse: 0.563206
[9663]	valid_0's rmse: 0.563198
[9664]	valid_0's rmse: 0.563196
[9665]	valid_0's rmse: 0.563193
[9666]	valid_0's rmse: 0.563189
[9667]	valid_0's rmse: 0.563189
[9668]	valid_0's rmse: 0.563188
[9669]	valid_0's rmse: 0.563188
[9670]	valid_0's rmse: 0.563186
[9671]	valid_0's rmse: 0.563175
[9672]	valid_0's rmse: 0.563156
[9673]	valid_0's rmse: 0.563151
[9674]	valid_0's rmse: 0.563148
[9675]	valid_0's rmse: 0.563129
[9676]	valid_0's rmse: 0.563125
[9677]	va

[9903]	valid_0's rmse: 0.561624
[9904]	valid_0's rmse: 0.56162
[9905]	valid_0's rmse: 0.561614
[9906]	valid_0's rmse: 0.561608
[9907]	valid_0's rmse: 0.561604
[9908]	valid_0's rmse: 0.5616
[9909]	valid_0's rmse: 0.561599
[9910]	valid_0's rmse: 0.561593
[9911]	valid_0's rmse: 0.561589
[9912]	valid_0's rmse: 0.561582
[9913]	valid_0's rmse: 0.561576
[9914]	valid_0's rmse: 0.561575
[9915]	valid_0's rmse: 0.561575
[9916]	valid_0's rmse: 0.561575
[9917]	valid_0's rmse: 0.561574
[9918]	valid_0's rmse: 0.561572
[9919]	valid_0's rmse: 0.561572
[9920]	valid_0's rmse: 0.561562
[9921]	valid_0's rmse: 0.561559
[9922]	valid_0's rmse: 0.561555
[9923]	valid_0's rmse: 0.561554
[9924]	valid_0's rmse: 0.561551
[9925]	valid_0's rmse: 0.561547
[9926]	valid_0's rmse: 0.561546
[9927]	valid_0's rmse: 0.561536
[9928]	valid_0's rmse: 0.561531
[9929]	valid_0's rmse: 0.561529
[9930]	valid_0's rmse: 0.561528
[9931]	valid_0's rmse: 0.561506
[9932]	valid_0's rmse: 0.561501
[9933]	valid_0's rmse: 0.561477
[9934]	vali

In [104]:
final_output['geoAltitude'] = final_test_null_DF['baroAltitude']
final_output

,id,latitude,longitude,geoAltitude
0,31,46.985571,6.236670,10965.18
1,79,47.413494,6.015626,11277.60
2,89,46.630148,8.871447,11582.40
3,101,49.264401,7.976657,11887.20
4,132,47.418800,8.491532,9014.46
...,...,...,...,...
109469,1836617,52.891362,9.745615,10363.20
109470,1836625,49.270687,3.406421,7886.70
109471,1836645,49.452413,7.327884,10668.00
109472,1836697,47.106710,11.068880,10713.72


In [105]:
LGB_submission_DF = final_output.copy()

LGB_submission_DF.to_csv('../output/sub_LGB_2.csv', index = False)
LGB_submission_DF

,id,latitude,longitude,geoAltitude
0,31,46.985571,6.236670,10965.18
1,79,47.413494,6.015626,11277.60
2,89,46.630148,8.871447,11582.40
3,101,49.264401,7.976657,11887.20
4,132,47.418800,8.491532,9014.46
...,...,...,...,...
109469,1836617,52.891362,9.745615,10363.20
109470,1836625,49.270687,3.406421,7886.70
109471,1836645,49.452413,7.327884,10668.00
109472,1836697,47.106710,11.068880,10713.72
